# 12 - Archivos y rutas con pathlib: organiza tu mundo en disco

## Objetivos de Aprendizaje

En esta sesion aprenderas:

1. Entender la diferencia entre rutas relativas y absolutas.
2. Crear y combinar rutas con `pathlib.Path`.
3. Inspeccionar partes de una ruta (nombre, extension, parent).
4. Verificar existencia y tipo de rutas.
5. Crear carpetas y archivos de forma segura.
6. Leer y escribir texto con codificacion.
7. Buscar archivos con patrones (`glob`, `rglob`).
8. Construir rutas relativas y normalizadas.
9. Resolver ejercicios practicos con archivos y rutas.

---

## Ruta de la sesion (secuencia ideal)

1. Que es una ruta y por que `pathlib`.
2. Crear rutas absolutas y relativas.
3. Partes y propiedades de una ruta.
4. Unir rutas y navegar carpetas.
5. Verificaciones y creacion segura.
6. Lectura y escritura de texto.
7. Busqueda con patrones.
8. Rutas relativas y normalizacion.
9. Ejercicios.
10. Resumen y errores comunes.


## 1. ?Que es una ruta y por que `pathlib`?

Una ruta (path) es la direccion que identifica un archivo o carpeta en el sistema de archivos.
Trabajar con rutas como strings es fragil: separadores distintos (`/` vs `\`), dobles barras y
concatenaciones con errores. `pathlib` ofrece un objeto `Path` que **entiende** la ruta y da
metodos claros para operar con ella.

Ventajas:
- Portabilidad entre Windows, macOS y Linux.
- Metodos expresivos (`exists`, `is_file`, `read_text`).
- Composicion segura con `/` en lugar de concatenar strings.


In [ ]:
from pathlib import Path

ruta = Path("data/ventas/enero.csv")
print(ruta)
print(type(ruta))


## 2. Rutas absolutas y relativas

- **Absoluta**: describe la ubicacion completa desde la raiz.
- **Relativa**: depende del directorio actual (CWD).

Usa `Path.cwd()` y `Path.home()` para orientarte. Con `resolve()` puedes normalizar.


In [ ]:
from pathlib import Path

print("CWD:", Path.cwd())
print("HOME:", Path.home())

ruta_rel = Path("data/archivo.txt")
print("Es absoluta?", ruta_rel.is_absolute())
print("Normalizada:", ruta_rel.resolve())


## 3. Partes de una ruta

`Path` te permite acceder facilmente a piezas comunes: nombre, extension, carpeta padre, etc.


In [ ]:
from pathlib import Path

ruta = Path("data/reportes/2024/enero.csv")
print("Nombre:", ruta.name)
print("Stem:", ruta.stem)
print("Extension:", ruta.suffix)
print("Partes:", ruta.parts)
print("Parent:", ruta.parent)


## 4. Unir rutas y navegar carpetas

Combina rutas con `/` o `joinpath()`. Evita concatenar strings. Tambien puedes cambiar
el nombre o la extension con `with_name()` y `with_suffix()`.


In [ ]:
from pathlib import Path

base = Path("data")
ruta = base / "raw" / "ventas.csv"
print(ruta)

copia = ruta.with_name("ventas_backup.csv")
print(copia)

parquet = ruta.with_suffix(".parquet")
print(parquet)


## 5. Verificar existencia y tipo

Antes de leer o escribir, confirma si una ruta existe y si es archivo o carpeta.


In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp:
    base = Path(tmp)
    archivo = base / "ejemplo.txt"
    archivo.write_text("hola", encoding="utf-8")

    print("Existe?", archivo.exists())
    print("Es archivo?", archivo.is_file())
    print("Es carpeta?", base.is_dir())


## 6. Crear carpetas y archivos

Usa `mkdir()` con `parents=True` para crear rutas anidadas y `exist_ok=True` para evitar errores
si ya existen. Para crear un archivo vacio, usa `touch()`.


In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp:
    base = Path(tmp)
    carpeta = base / "data" / "raw"
    carpeta.mkdir(parents=True, exist_ok=True)

    archivo = carpeta / "nuevo.txt"
    archivo.touch()

    print(carpeta.exists(), archivo.exists())


## 7. Leer y escribir texto

`read_text()` y `write_text()` son utiles para archivos pequenos. Siempre especifica `encoding`.


In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp:
    ruta = Path(tmp) / "nota.txt"
    ruta.write_text("Linea 1
Linea 2
", encoding="utf-8")

    contenido = ruta.read_text(encoding="utf-8")
    lineas = contenido.splitlines()
    print(lineas)


## 8. `open()` para archivos grandes

Cuando el archivo es grande, lee linea por linea con `open()` y un context manager.


In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp:
    ruta = Path(tmp) / "grande.txt"
    ruta.write_text("a
b
c
", encoding="utf-8")

    with ruta.open("r", encoding="utf-8") as f:
        for linea in f:
            print(linea.strip())


## 9. Buscar archivos con `glob` y `rglob`

`glob()` busca en una carpeta; `rglob()` busca de forma recursiva. Los patrones usan `*` y `?`.


In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp:
    base = Path(tmp)
    (base / "docs").mkdir()
    (base / "data").mkdir()
    (base / "docs" / "a.txt").write_text("x", encoding="utf-8")
    (base / "data" / "b.csv").write_text("x", encoding="utf-8")
    (base / "data" / "c.csv").write_text("x", encoding="utf-8")

    csvs = list(base.rglob("*.csv"))
    print([p.name for p in csvs])


## 10. Rutas relativas, normalizacion y portabilidad

- `relative_to()` calcula una ruta relativa desde un base.
- `as_posix()` convierte a formato con `/` (util para logs y URLs).
- `resolve()` normaliza y elimina `..` o `.`.


In [ ]:
from pathlib import Path

base = Path("proyecto")
archivo = base / "data" / "raw" / "ventas.csv"

relativa = archivo.relative_to(base)
print(relativa)
print(relativa.as_posix())


## 11. Buenas practicas

- Evita concatenar rutas con strings.
- Especifica `encoding` al leer/escribir.
- Usa `exist_ok=True` si el directorio puede existir.
- Prefiere `Path` sobre `os.path` para legibilidad.
- Valida rutas de entrada si vienen del usuario.


## 12. Ejercicios Practicos

Resuelve los siguientes ejercicios. Cada uno incluye una propuesta de solucion.


### Ejercicio 1: Normalizar ruta de usuario
**Tarea**: Dada la cadena `texto = "~/proyecto/../proyecto/data/./ventas.csv"`, crea un `Path`
absoluto y normalizado. Luego imprime la ruta, su `parent` y su `name`. No uses `os.path`.


In [ ]:
# Tu codigo aqui:
# texto = "~/proyecto/../proyecto/data/./ventas.csv"
# ...

# SOLUCION:
from pathlib import Path

texto = "~/proyecto/../proyecto/data/./ventas.csv"
ruta = Path(texto).expanduser().resolve()
print(ruta)
print("Parent:", ruta.parent)
print("Name:", ruta.name)


### Ejercicio 2: Metadatos de una ruta
**Tarea**: Para `ruta = Path("data/reportes/2024/enero.csv")`, crea un diccionario con:
`nombre`, `extension`, `carpeta` (padre) y `profundidad` (numero de partes).


In [ ]:
# Tu codigo aqui:
# ruta = Path("data/reportes/2024/enero.csv")
# ...

# SOLUCION:
from pathlib import Path

ruta = Path("data/reportes/2024/enero.csv")
info = {
    "nombre": ruta.name,
    "extension": ruta.suffix,
    "carpeta": ruta.parent.name,
    "profundidad": len(ruta.parts),
}
print(info)


### Ejercicio 3: Crear estructura de proyecto
**Tarea**: Crea la estructura `mi_proyecto/data/raw`, `mi_proyecto/data/processed` y
`mi_proyecto/reports`. La solucion no debe fallar si ya existen.


In [ ]:
# Tu codigo aqui:
# base = Path("mi_proyecto")
# ...

# SOLUCION:
from pathlib import Path

base = Path("mi_proyecto")
(base / "data" / "raw").mkdir(parents=True, exist_ok=True)
(base / "data" / "processed").mkdir(parents=True, exist_ok=True)
(base / "reports").mkdir(parents=True, exist_ok=True)


### Ejercicio 4: Filtrar rutas
**Tarea**: A partir de la lista `rutas`, filtra solo los `.csv` dentro de `data/raw`
y ordenalos por nombre.


In [ ]:
# Tu codigo aqui:
# rutas = [
#     "data/raw/ventas_enero.csv",
#     "data/raw/ventas_feb.csv",
#     "data/processed/ventas_mar.csv",
#     "data/raw/notas.txt",
#     "data/raw/.gitkeep",
# ]
# ...

# SOLUCION:
from pathlib import Path

rutas = [
    "data/raw/ventas_enero.csv",
    "data/raw/ventas_feb.csv",
    "data/processed/ventas_mar.csv",
    "data/raw/notas.txt",
    "data/raw/.gitkeep",
]

filtradas = [
    Path(r)
    for r in rutas
    if Path(r).suffix == ".csv" and Path(r).parts[:2] == ("data", "raw")
]

filtradas = sorted(filtradas, key=lambda p: p.name)
print([p.as_posix() for p in filtradas])


### Ejercicio 5: Renombrar con snake_case
**Tarea**: Convierte `reportes/Resumen Enero 2024.txt` en `reportes/resumen_enero_2024.md`
sin cambiar la carpeta. Usa `with_name()` y `with_suffix()`.


In [ ]:
# Tu codigo aqui:
# ruta = Path("reportes/Resumen Enero 2024.txt")
# ...

# SOLUCION:
from pathlib import Path

ruta = Path("reportes/Resumen Enero 2024.txt")
nombre = ruta.stem.lower().replace(" ", "_")
nueva = ruta.with_name(nombre).with_suffix(".md")
print(nueva)


### Ejercicio 6: Rutas relativas para logs
**Tarea**: Dada una base `proyecto`, genera una lista de rutas relativas en formato posix.


In [ ]:
# Tu codigo aqui:
# base = Path("proyecto")
# archivos = [
#     base / "data/raw/ventas.csv",
#     base / "data/raw/clientes.csv",
#     base / "reports/resumen.txt",
# ]
# ...

# SOLUCION:
from pathlib import Path

base = Path("proyecto")
archivos = [
    base / "data/raw/ventas.csv",
    base / "data/raw/clientes.csv",
    base / "reports/resumen.txt",
]

relativas = [p.relative_to(base).as_posix() for p in archivos]
print(relativas)


### Ejercicio 7: Contar archivos por extension
**Tarea**: Con la lista `rutas`, crea un diccionario con el conteo por extension
(ignora archivos sin extension). Usa minusculas para comparar.


In [ ]:
# Tu codigo aqui:
# rutas = [
#     "data/raw/ventas.csv",
#     "data/raw/clientes.CSV",
#     "data/raw/readme",
#     "docs/guia.md",
#     "docs/intro.MD",
# ]
# ...

# SOLUCION:
from pathlib import Path

rutas = [
    "data/raw/ventas.csv",
    "data/raw/clientes.CSV",
    "data/raw/readme",
    "docs/guia.md",
    "docs/intro.MD",
]

conteo = {}
for r in rutas:
    suf = Path(r).suffix.lower()
    if not suf:
        continue
    conteo[suf] = conteo.get(suf, 0) + 1

print(conteo)


### Ejercicio 8: Top 3 archivos mas grandes
**Tarea**: En una carpeta real (por ejemplo `Path.cwd()`), encuentra los 3 archivos mas
grandes y muestra su nombre y tamano en bytes.


In [ ]:
# Tu codigo aqui:
# carpeta = Path.cwd()
# ...

# SOLUCION:
from pathlib import Path

carpeta = Path.cwd()
archivos = [p for p in carpeta.rglob("*") if p.is_file()]
top3 = sorted(archivos, key=lambda p: p.stat().st_size, reverse=True)[:3]

for p in top3:
    print(p.name, p.stat().st_size)


## 13. Resumen de conceptos clave

| Concepto | Que es | Ejemplo |
|----------|--------|---------|
| `Path` | Objeto que representa una ruta | `Path("data/archivo.txt")` |
| Ruta absoluta | Incluye la raiz | `Path.cwd()` |
| Ruta relativa | Depende del CWD | `Path("data/archivo.txt")` |
| `parent` | Carpeta padre | `ruta.parent` |
| `suffix` | Extension del archivo | `ruta.suffix` |
| `glob/rglob` | Busqueda por patrones | `base.rglob("*.csv")` |
| `read_text` | Leer texto completo | `ruta.read_text()` |
| `write_text` | Escribir texto completo | `ruta.write_text("hola")` |


## 14. Errores comunes y buenas practicas

1. Concatenar rutas con strings en lugar de `Path`.
2. No especificar `encoding` al leer/escribir.
3. Usar `resolve()` sin entender que normaliza `..` y `.`.
4. Olvidar `exist_ok=True` al crear carpetas.
5. Asumir que una ruta existe sin verificar `exists()`.
